# Context Free Grammar Parsing using the CYK Algorithm
## Muya Guoji and Evelyn Kessler
## December 17, 2023

Outline

A 10-15 minutes presentation during final events period
Code for your implementation (when relevant) and instructions for how to run the code
A 5 pages write up describing the work and walking through the code

- introduction; what is the CYK algorithm, our inspiration for the project, basic premise of CFG parsing
- introducing the CYK algorithm; what is Chomsky Normal Form (unary terminal and binary non-terminals) and why do we need it/how to write in it, basic architecture of the algorithm, possible uses/real world examples, theory walkthrough with lots of visuals (parsing trees)
- step by step; detailed step by step through the algorithm (practical walkthrough), explain each line, use print outs or other visuals to explain what the loops are doing
- using CYK parsing on natural language; transformers fail to get semantics while parsing can clarify ambiguous language, what are parts of speech, examples with real sentences (both parsing tree and run through our algorithm)

- showing our algorithm with comments/docstring/etc
- short paragraph of how to run it what the print outs mean
- some example CFGs that you can run and let user pick the sentence (try and create true and false sentences)

### Table of Contents
1. [Introduction](#introduction) - Evelyn
    1. [What is CFG Parsing?](#subintroduction1) - Evelyn
    2. [CFG Parsing in the Real World](#subintroduction2) - Evelyn
2. [The CYK Algorithm](#paragraph1)
    1. [Chomsky Normal Form](#subparagraph11) - Muya
    2. [Converting CFGs into Chomsky Normal Form (CNF)](#subparagraph12) - Evelyn
    2. [Basic Code Architecture](#subparagraph13) - Muya
    3. [Uses in the World](#subparagraph14) - Evelyn
    4. [Visual Walkthrough](#subparagraph15) - Evelyn
3. [Code Walkthrough](#paragraph2) - Muya

4. [CYK Parsing on Natural Langauge](#paragraph3)
    1. [Benefits of CYK over Transformers](#subparagraph31) - Muya
    2. [What are Parts of Speech?](#subparagraph32) - Muya
    3. [Writing CNF CFGs for Natural Language Parsing](#subparagraph33) - Evelyn
    4. [Visual Walkthrough: Parsing a Sentence](#subparagraph34) - Evelyn
5. [Appendix](#appendix)
    1. [CYK Implementation](#subappendix1)
    2. [Instructions for Running the Algorithm](#subappendix2)
    3. [Example CFGs and strings to parse](#subappendix3)
    4. [How to create your own CFG and strings to parse](#subappendix4)
6. [Sources](#sources)

#### Introduction <a name="introduction"></a>
A context free grammar, or CFG, is a grammar that is used to generate all possible strings in a given language. A context free grammar, G, can be defined by a tuple 

$$
G = (\Sigma, \Gamma, R, S)
$$

where $\Sigma$ is a list of terminal symbols, $\Gamma$ is a list of non-terminal symbols, R is a list of rules of the form $x \rightarrow \mu$ where $\mu \in (\Gamma \cup \Sigma)*$, and $S$ is the start symbol. One way of defining a language is to write a CFG for that language. Then, we can find for any string whether that string is in the language by evaluating whether the rules of the CFG could possibly generate that string. For this method we start with the start symbol $S$ and apply the rules continuously until we generate the desired string. If we can't generate the string after (a reasonable number of) rules applications *double check this is actually right; are CFGs where you get stuck or where you run out??*, we can conclude that the given string is not valid for the language described by that CFG.

##### What is CFG Parsing? <a name="subintroduction1"></a>
CFG parsing is another way to find whether a given string is valid for a language as described by a CFG. Instead of starting with S and building up to the given string using the rules, CFG parsing works backwards, starting with the given string and seeing if we can reduce it to S using a combination of the given rules.

CFG parsing is a method used to determine if a given string conforms to the rules of a specific language as defined by its CFG. Unlike the previous CFG method, which begins with the start symbol and attempts to construct the string by applying grammar rules, CFG parsing starts with the given string and works backwards, trying to decompose it into its basic elements until it reaches the start symbol. This process involves applying the CFG rules in reverse.

CFG parsing can be implemented with several different strategies and algorithms. These include top-down parsing methods like recursive descent parsing, where the parser starts at the highest level of the grammar and works its way down, and bottom-up methods like shift-reduce parsing, which start with the input and gradually transform it into the syntax tree of the grammar.

The essential part of CFG parsing is the construction of a parse tree or syntax tree. This tree represents the syntactic structure of the string according to the grammar rules. Each node in the tree corresponds to a grammar rule, and the leaves represent the elements of the string. By analyzing the tree, we can understand how the string is structured according to the grammar and whether it's a valid construct in the language.

CFG parsing is crucial in various applications because it allows for the automatic analysis of the structure of strings, which is fundamental in language processing, programming languages, and other computational linguistics areas. It enables computers to understand and manipulate complex structures in human and programming languages, making it a cornerstone of modern computing.

##### CFG Parsing in the Real World <a name="subintroduction2"></a>
CFG parsing is valuable in many practical scenarios, particularly in computer science and linguistics. A prime example is in natural language processing (NLP), where CFG parsing helps computers make sense of human language. It breaks down sentences into grammatical parts, which is crucial for things like translating languages since computers need to get the structure right to translate words accurately from one language to another. Another key use of CFG parsing is in building compilers for programming languages. Compilers use CFG to dissect the code that programmers write, turning it into something a computer can understand and act on.

Besides these, CFG parsing finds its place in various other areas such as:
- Checking code for errors in programming tools.
- Helping voice recognition systems understand what we say.
- Improving writing software by checking for grammatical mistakes.
- Analyzing complex mathematical formulas in scientific programs.



#### The CYK Algorithm <a name="paragraph1"></a>

##### Chomsky Normal Form <a name="subparagraph11"></a>
Before delving into the algorithm, we would like to first touch on the concept of Chomsky Normal Form (CNF). It is a prerequisite for understanding the CYK algorithm, as the algorithm specifically requires grammars to be in CNF in order to be processed. 

CNF, developed by the well-known modern linguist Noam Chomsky, aims to simplify the rules of context-free
grammars for more efficient parsing and algorithmic analysis. It is a specific method for expressing CFGs, wherein every production rule takes one of two forms: either a non-terminal symbol producing two other non-terminal symbols or a non-terminal symbol producing a single terminal symbol. This single terminal symbol may include epsilon, indicating the deletion of a sentence. 

$$
A \rightarrow BC
$$
$$
A \rightarrow a
$$
$$
S \rightarrow \Epsilon
$$
*(Uppercase letters represent non-terminal symbols, and lowercase letters represent terminal symbols)*


By unifying production rules in such a specific and clean form really simplifies the design and analysis of parsing
algorithms like CYK. 

##### Converting CFGs into Chomsky Normal Form (CNF) <a name="subparagraph33"></a>

In CNF, each rule must either produce two non-terminal symbols or a single terminal symbol. Consider the grammars below.

$$G1 = S \rightarrow a, S \rightarrow AZ, A \rightarrow a, Z \rightarrow z$$
$$G2 = S \rightarrow a, S \rightarrow aZ, Z \rightarrow a$$

Grammar $G1$ is in CNF since the first, third, and fourth rules all produce a single terminal symbol and the second rule produces two non-terminal symbols. Grammar $G2$ is not in CNF since the rule $S \rightarrow aZ$ contains a terminal symbol ($a$) followed by a non-terminal symbol ($Z$).

To convert a CFG to CNF, follow these rules:

1. **Eliminate the start symbol from right hand side (RHS)**: If there is a start symbol S in the RHS of any rule, create a new rule $S0 \rightarrow S$ and replace the $S$ in the RHS with $S0$.

2. **Eliminate Null Productions**: Remove any rules that produce an empty string, replacing them with alternative productions that achieve the same language representation without the null option.

3. **Eliminate Unit Productions**: Replace rules that produce a single non-terminal with rules that produce terminals directly. For instance $A \rightarrow B$ and $B \rightarrow a$ can be replaced with $A \rightarrow a$.

4. **Ensure all rules with terminals in RHS have a single terminal**: Eliminate terminals from the RHS if they are with other terminals or non-terminals by decomposing into multiple rules. For example, if you have the rule $X \rightarrow xY$ you can decompose it into $X \rightarrow ZY$ and $Z \rightarrow x$.

5. **Ensure all rules with non-terminals in RHS have exactly two non-terminals** : Eliminate rules with more than two non-terminal symbols by decomposing them into multiple rules. For example, the rule $X \rightarrow XYZ$ can be broken into $X \rightarrow PZ$ and $P \rightarrow XY$.

Let's try an example. The grammar $S \rightarrow \epsilon, S \rightarrow aSb$ generates the language $a^nb^n | n \geq 0$.

1. Eliminate start symbols from the right hand side.
We can eliminate the S from the RHS by writing a new rule $S0 \rightarrow S$ where S0 is our new start symbol.

$$S \rightarrow \epsilon$$
$$S \rightarrow aSb$$
$$S0 \rightarrow S$$

2. Eliminate Null Productions
Let's remove the epsilon and add a rule to cover the case where the $S \rightarrow \epsilon$ would be used. If this rule were used, the $\epsilon$ would replace the $T$ in $S \rightarrow aTb$ leaving just the $a$ and $b$. We can replicate this behavior by declaring $S \rightarrow ab$. Note that this does not cover the case where $n=0$. To cover this case, we need to add back in a rule $S \rightarrow \epsilon$.

$$S \rightarrow ab$$
$$S \rightarrow aSb$$
$$S \rightarrow \epsilon$$
$$S0 \rightarrow S$$

3. Eliminate Unit Productions
We have one unit production in our grammar, $S0 \rightarrow S$. We can fix this by explicitely defining what $S0$ can go to as $S0 \rightarrow ab | aSb | \epsilon$. Note that $S0$ can go to $\epsilon$ to create the empty string, but $S$ no longer goes to $\epsilon$ since we've replaced that case.

$$S \rightarrow ab | aSb$$
$$S0 \rightarrow ab | aSb | \epsilon$$

4. Ensure all rules with terminals in RHS have a single terminal
The rules $S \rightarrow aSb$$ and $$S \rightarrow ab$ have multiple terminals and/or terminals with non-terminals, as do their $S0$ equivalents. We can decompose them by introducing new non-terminals, $A \rightarrow a$$ and $$B \rightarrow b$.

$$S \rightarrow AB | ASB$$
$$S0 \rightarrow AB | ASB | \epsilon$$
$$A \rightarrow a$$
$$B \rightarrow b$$

5. Ensure all rules with non-terminals in RHS have exactly two non-terminals
The rules $S \rightarrow ASB$ and $S0 \rightarrow ASB$ have three non-terminals. We can introduce a new non-terminal to split the rule into $S \rightarrow AR$ and $R \rightarrow SB$ and same for $S0$.

$$S \rightarrow AB | AR$$
$$S0 \rightarrow AB | AR | \epsilon$$
$$R \rightarrow SB$$
$$A \rightarrow a$$
$$B \rightarrow b$$

After covering the five steps, we've converted our CFG into CNF! Note that this is not the only possible CNF for this language, there are many options that will work.
Let's double check that our CNF CFG makes sense for the language $a^nb^n | n \geq 0$ using our parser!

In [4]:
from CYKalgorithm import CYK_parse

grammar_anbn = {
    'S': {'AB', 'AR', 'e'}, #S: start sym. note that we've replaced S0 with S and S with T.
    'T': {'AB', 'AR'},
    'R': {'TB'},
    'A': {'a'},
    'B': {'b'}
}

# Example strings (TRUE)
string_anbn = "aabb"
string1_abnb = ""
string2_anbn = "ab"

# Example strings (FALSE)
string3_anbn = "abbb"
string4_anbn = "b"
string5_anbn = "abab"

table, result = CYK_parse(grammar_anbn, string5_anbn) #change the string entry to other example string#_anbn to see different results, or add your own string!
print("Can the string be derived?", result)

a found in ('A', {'a'}). Adding A to T[0][0]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
b found in ('B', {'b'}). Adding B to T[1][1]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'B'}, set(), set()]
[set(), set(), set(), set()]
[set(), set(), set(), set()]
a found in ('A', {'a'}). Adding A to T[2][2]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'B'}, set(), set()]
[set(), set(), {'A'}, set()]
[set(), set(), set(), set()]
b found in ('B', {'b'}). Adding B to T[3][3]
Parsing table:
[{'A'}, set(), set(), set()]
[set(), {'B'}, set(), set()]
[set(), set(), {'A'}, set()]
[set(), set(), set(), {'B'}]
A found in T[0][0] and B found in T[1][1]. S : AB is valid. Adding S to T[0][1]
Parsing table:
[{'A'}, {'S'}, set(), set()]
[set(), {'B'}, set(), set()]
[set(), set(), {'A'}, set()]
[set(), set(), set(), {'B'}]
A found in T[0][0] and B found in T[1][1]. T : AB is valid. Adding T to T[0][1]
Parsi

##### Basic Code Architecture <a name="subparagraph12"></a>
Let's begin the code run-through of the algorithm with a high-level overview of the CYK algorithm's architecture:

1. Input and Grammar Preparation: The algorithm starts with an input string, which are sentences breaking down into words, and a set of grammar rules in Chomsky Normal Form (CNF).

2. Table Initialization: A table (matrix) is created with dimensions based on the length of the input string. This table will be used to store possible grammar derivations for substrings of the input.

3. Table Filling: starting from the bottom level, the algorithm fills in the table and moves upward gradually. Each cell in the table represents a substring of the sentence and is filled with all possible grammar rules that could generate that substring. 

4. Combining Substrings: As it moves up the table, the algorithm combines smaller substrings that have already been matched with grammar rules to form larger substrings. It checks to see if these larger substrings can be generated by any of the CFG rules.

5. Final Verification: Once the table is filled, the algorithm checks the top cell , which represents the entire string (sentence). If this cell contains the start symbol of the grammar, the string is considered derivable from the given grammar.

6. Result and Visualization: Visualization of the table after each iteration can be used to enhance clarity and explainability of the processes for model implementers.

##### Uses in the World <a name="subparagraph13"></a>
The CYK algorithm has several practical applications in the real world and is one of the most common CFG parsing algorithms. In computational linguistics, the CYK algorithm is used for parsing natural language sentences. It helps in syntactic analysis, which is fundamental for tasks like sentiment analysis (determining emotional tone behind a body of text), information extraction (finding specific information in a body of text), and automated question answering. This application is crucial for developing sophisticated AI chatbots and virtual assistants that can understand and respond to human language effectively.

In the realm of programming, the CYK algorithm is employed in the development of compilers and interpreters for programming languages. Its ability to parse context-free grammars makes it useful for syntax analysis, ensuring that the code written by programmers adheres to the grammatical rules of the programming language. This aspect is vital for error detection and correction during software development.

Additionally, in more specialized areas, the CYK algorithm can be applied in DNA sequencing and bioinformatics for analyzing and interpreting the genetic information contained in DNA sequences, where the algorithm helps in understanding the complex structures and patterns within genetic data. 

As we can see, the CYK algorithm has many uses in practical scenarios in computer science, math, technology, and even biology and other fields.

##### Visual Walkthrough <a name="subparagraph14"></a>

#### Code Walkthrough <a name="paragraph2"></a>

#### CYK Parsing on Natural Langauge<a name="paragraph3"></a>

##### The Benifits of CYK in NLP and a Comparative Analysis with Transformers <a name="subparagraph31"></a>
Natural Language Processing (NLP) is a field is an interdisciplinary subfield of computer science and linguistics. that is primarily concerned with giving computers the ability to support and manipulate human language (*Wikipedia*). It encompasses a range of algorithmic tools for enabling computers to understand and process human language. Among these tools are transformers and the CYK algorithm.

With the development and hype surrounding many large language models, transformers have garnered significant worldwide attention, possibly a byproduct of their attention mechanisms. While transformers are powerful tools in NLP, their approach to understanding language is based on statistical learning from large datasets. They are highly effective in many contexts *but might not always align perfectly with the complexities of human language understanding*. On the other hand, the CYK algorithm, with its basis in formal grammar theory, offers a more rule-based approach to parsing language. It can break down sentences into their grammatical components, making CYK particularly valuable in applications where precise syntax parsing is crucial, such as in compiling programming languages or in certain aspects of natural language understanding where the exact structure of sentences is significant. 

Which brings us to the next topic: parts of speech. 

##### Part of Speech <a name="subparagraph32"></a>
In NLP and any linguistics domain, parts of speech is a key terminology. Part of speech refers to a category of words with similar grammatical properties. Words in the same part of speech play similar roles in sentences and share similar rules of grammar. The main parts of speech in English include nouns, pronouns, verbs, adjectives, prepositions and so on. Identifying parts of speech and aligning substrings with these parts are the main preparations to enable the CYK algorithm to analyze sentence structure. 

For instance, with a suitable context-free grammar provided in Chomsky Normal Form (CNF) that shows 'blue' as an adjective, 'bird' as a noun, 'sings' as a verb, 'happily' as an adverb, 'in' as a preposition, 'the' as a definite article, and 'spring' as a noun, the CYK algorithm can determine whether a sentence like 'The blue bird sings happily in the spring' is structurally valid according to that grammar.

##### Writing CNF CFGs for Natural Language Parsing <a name="subparagraph33"></a>

##### Visual Walkthrough: Parsing a Sentence <a name="subparagraph34"></a>

#### Sources <a name="sources"></a>
https://jayd.ml/algorithms/chomsky.html
https://www.geeksforgeeks.org/converting-context-free-grammar-chomsky-normal-form/
https://www.geeksforgeeks.org/what-is-context-free-grammar/
https://www.geeksforgeeks.org/cyk-algorithm-for-context-free-grammar/